In [2]:
import os
import tensorflow as tf
import numpy as np
import math
import timeit
#import matplotlib.pyplot as plt
import CNet
#%matplotlib inline

In [3]:
USE_GPU = True

if USE_GPU:
    device = '/device:GPU:0'
else:
    device = '/cpu:0'

# Constant to control how often we print when training models
print_every = 100

print('Using device: ', device)

Using device:  /device:GPU:0


In [4]:
def conv_centralnet_test():
    tf.reset_default_graph()

    with tf.device(device):
        #conv1
        x1 = tf.placeholder(tf.float32)
        conv1_w1 = tf.zeros((5, 5, 3, 6))
        conv1_b1 = tf.zeros((6,))
        conv1_w2 = tf.zeros((3, 3, 6, 9))
        conv1_b2 = tf.zeros((9,))
        fc1_w = tf.zeros((32 * 32 * 9, 10))
        fc1_b = tf.zeros((10,))
        feed1 = [conv1_w1, conv1_b1, conv1_w2, conv1_b2, fc1_w, fc1_b]
        
        #conv2
        
        x2 = tf.placeholder(tf.float32)
        conv2_w1 = tf.zeros((5, 5, 3, 6))
        conv2_b1 = tf.zeros((6,))
        conv2_w2 = tf.zeros((3, 3, 6, 9))
        conv2_b2 = tf.zeros((9,))
        fc2_w = tf.zeros((32 * 32 * 9, 10))
        fc2_b = tf.zeros((10,))
        feed2 = [conv2_w1, conv2_b1, conv2_w2, conv2_b2, fc2_w, fc2_b]

        
        #centralnet
        c11 = tf.zeros((32*32*6, 4))
        c12 = tf.zeros((32*32*6, 4))
        c21 = tf.zeros((32*32*9, 10))
        c22 = tf.zeros((32*32*9, 10))
        cc1 = tf.zeros((4, 10))
 
        feedc=[c11,c12,c21,c22,cc1]

        #final
        
        params=[feed1,feed2,feedc]
        scores=CNet.conv_centralnet(x1,x2,params)
    # Inputs to convolutional layers are 4-dimensional arrays with shape
    # [batch_size, height, width, channels]
    x1_np = np.zeros((64, 32, 32, 3))
    x2_np = np.zeros((64, 32, 32, 3))   
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        scores_np = sess.run(scores, feed_dict={x1: x1_np,x2:x2_np})
        print('scores_np has shape: ', scores_np.shape)

with tf.device('/cpu:0'):
    conv_centralnet_test()

scores_np has shape:  (64, 10)


In [5]:
import GetCifar

# Invoke the above function to get our data.
NHW = (0, 1, 2)
X_train, y_train, X_val, y_val, X_test, y_test = GetCifar.load_cifar10()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

train_dset = GetCifar.Dataset(X_train, y_train, batch_size=64, shuffle=True)
val_dset = GetCifar.Dataset(X_val, y_val, batch_size=64, shuffle=False)
test_dset = GetCifar.Dataset(X_test, y_test, batch_size=64)

# We can iterate through a dataset like this:
for t, (x, y) in enumerate(train_dset):
    print(t, x.shape, y.shape)
    if t > 5: break

Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,) int32
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)
0 (64, 32, 32, 3) (64,)
1 (64, 32, 32, 3) (64,)
2 (64, 32, 32, 3) (64,)
3 (64, 32, 32, 3) (64,)
4 (64, 32, 32, 3) (64,)
5 (64, 32, 32, 3) (64,)
6 (64, 32, 32, 3) (64,)


In [11]:
import TR
def train_part2(model_fn, init_fn, learning_rate):

    # First clear the default graph
    tf.reset_default_graph()
    is_training = tf.placeholder(tf.bool, name='is_training')
    # Set up the computational graph for performing forward and backward passes,
    # and weight updates.
    with tf.device(device):
        # Set up placeholders for the data and labels
        x1 = tf.placeholder(tf.float32, [None, 32, 32, 3])
        x2 = tf.placeholder(tf.float32, [None, 32, 32, 3])
        
        y = tf.placeholder(tf.int32, [None])
        params = init_fn()           # Initialize the model parameters
        scores = model_fn(x1,x2, params) # Forward pass of the model
        loss = TR.training_step(scores, y, params, learning_rate)

    # Now we actually run the graph many times using the training data
    with tf.Session() as sess:
        # Initialize variables that will live in the graph
        sess.run(tf.global_variables_initializer())
#########################################################################
        for t, (x_np, y_np) in enumerate(train_dset):
            # Run the graph on a batch of training data; recall that asking
            # TensorFlow to evaluate loss will cause an SGD step to happen.
            feed_dict = {x1: x_np,x2:x_np, y: y_np}
            loss_np = sess.run(loss, feed_dict=feed_dict)
           
            # Periodically print the loss and check accuracy on the val set
            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss_np))
                TR.check_accuracy(sess, val_dset, x1,x2, scores, is_training)
################################################################################

learning_rate = 1e-2
train_part2(CNet.conv_centralnet, TR.conv_centralnet_init, learning_rate)

Iteration 0, loss = 5.1625
Got 131 / 1000 correct (13.10%)
Iteration 100, loss = 2.2713
Got 373 / 1000 correct (37.30%)
Iteration 200, loss = 1.3442
Got 474 / 1000 correct (47.40%)
Iteration 300, loss = 1.6089
Got 484 / 1000 correct (48.40%)
Iteration 400, loss = 1.6382
Got 500 / 1000 correct (50.00%)
Iteration 500, loss = 1.4475
Got 535 / 1000 correct (53.50%)
Iteration 600, loss = 1.3655
Got 559 / 1000 correct (55.90%)
Iteration 700, loss = 1.4740
Got 540 / 1000 correct (54.00%)


In [31]:
import provider
data_input=provider.DVR_Provider()
imgs, labels = data_input.load_one_batch(32 ,"train")
print(imgs.shape)
print(labels.shape)

['data/demo/DVR/2065.jpg' 'data/demo/DVR/803.jpg' 'data/demo/DVR/433.jpg'
 ... 'data/demo/DVR/1122.jpg' 'data/demo/DVR/3608.jpg'
 'data/demo/DVR/275.jpg']
(32, 66, 200, 3)
(32, 2)
